In [1]:
import json
import folium
import requests
from data_keys import ScoringKeys as SK, LocationKeys as LK, CoordinateKeys as CK
from api import getGeneralData, getMapData, getGame
from dotenv import load_dotenv
import os

# Load Game <br>
Enter one of the game id stored in folder my_games

In [2]:
load_dotenv()
apiKey = os.environ["apiKey"]
id_ = input("Enter game id: ")

In [3]:
game = {}
print(f"1: Fetch local game")
print(f"2: Fetch game from Considtion")
option = input("1 for Local game or 2 for Considtion game: ")

match option:
        case "1":
                with open(f"my_games\{id_}.json", 'r', encoding="utf8") as f:
                        game = json.load(f)
        case "2":
                game = getGame(id_)
        case _:
                print("Invalid choice.")


1: Fetch local game
2: Fetch game from Considtion


# Load Map and General data

In [4]:
##Get map data from Considition endpoint
mapName = game[SK.mapName]
mapEntity = getMapData(mapName, apiKey)

##Get non map specific data from Considition endpoint
generalData = getGeneralData()

https://api.considition.com


# Create map <br>
Generates a simple visualization for a selected game. <br>
Green circle: Location where one or more refill stations where placed. (Click on the circle for a pop up with info) <br>
Blue circle: Location where no refill stations where placed. (Click on the circle for a pop up with info) <br>
Red circle: Visual representation of footfall.

In [5]:
MAP_CENTER = [
    (mapEntity["border"]["latitudeMax"] + mapEntity["border"]["latitudeMin"]) / 2,
    (mapEntity["border"]["longitudeMax"] + mapEntity["border"]["longitudeMin"]) / 2,
]

map_folium = folium.Map(
    location=MAP_CENTER,
    zoom_start=12,
    tiles="cartodb positron",
    width=1200,
    height=580,
)

for hot_spot in mapEntity["hotspots"]:
    folium.Circle(
        [hot_spot[CK.latitude], hot_spot[CK.longitude]],
        radius=hot_spot["spread"],
        opacity=0.2,
        fill_opacity=0.4,
        color="#FF5959",
        fill_color="#FF5959",
    ).add_to(map_folium)

for key in mapEntity[LK.locations]:
    if key not in game[LK.locations]:
        loc = mapEntity[LK.locations][key]
        t = loc[LK.locationType]
        salesVol = loc[LK.salesVolume]
        footfall = loc[LK.footfall]
        folium.CircleMarker(
            [
                mapEntity[LK.locations][key][CK.latitude],
                mapEntity[LK.locations][key][CK.longitude],
            ],
            radius=3,
            color="#0080bb",
            fill_color="#0080bb",
        ).add_child(
            folium.Popup(
                f"name: {key} <br> locationType: {t} <br> Sales: {salesVol} <br> footfall: {footfall}",
                min_width=300,
                max_width=300,
            )
        ).add_to(
            map_folium
        )

for key in game[LK.locations]:
    loc = game[LK.locations][key]
    t = loc[LK.locationType]
    ftf = loc[LK.footfall]
    fs9 = loc[LK.f9100Count]
    fs3 = loc[LK.f3100Count]
    sv = loc[LK.salesVolume]
    sc = loc[LK.salesCapacity]
    lc = loc[LK.leasingCost]
    er = loc[SK.earnings]
    folium.CircleMarker(
        [
            mapEntity[LK.locations][key][CK.latitude],
            mapEntity[LK.locations][key][CK.longitude],
        ],
        radius=3,
        color="#32a852",
        fill_color="#32a852",
    ).add_child(
        folium.Popup(
            f"name: {key} <br> Location type: {t} <br> Nr freestyle9100: {fs9} <br> Nr freestyle3100: {fs3} <br> Sales efter scoring: {sv} <br> Refill capacity: {sc} <br> Leasing cost: {lc} <br> Footfall: {ftf} <br> Earnings: {er}",
            min_width=300,
            max_width=300,
        )
    ).add_to(
        map_folium
    )

print(game[SK.gameScore])
map_folium

{'kgCo2Savings': 2414.0, 'totalFootfall': 17.044106241060305, 'earnings': 541034.0, 'total': 9808647.0}
